In [ ]:
import torch
import numpy as np
import pickle
import pandas as pd
import os


In [ ]:
!git clone https://github.com/anthony-frion/ShallowWaterJAX

Cloning into 'ShallowWaterJAX'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 32 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 8.58 MiB | 7.32 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
def load_model(model, filename='my_checkpoint.pth.tar', device='cpu'):
    # with torch.no_grad():
    assert os.path.isfile(filename), f"Error: {filename} not found"
    if os.path.isfile(filename):
        print("DEVICE on Loading model: ", device)
        checkpoint = torch.load(filename, map_location=torch.device(device))
        # checkpoint = torch.load(filename, map_location=torch.device('cuda'))
        # print("=> Loading model:", checkpoint)
        print("+++++++++++++++++++ LOADING trained model ++++++++++++++++++++++++++")
        model.load_state_dict(checkpoint['model_state_dict'], weight_)
        return model

def get_checkpoint(filename='my_checkpoint.pth.tar', device='cpu'):
  # with torch.no_grad():
    assert os.path.isfile(filename), f"Error: {filename} not found"
    if os.path.isfile(filename):
        checkpoint = torch.load(filename, map_location=torch.device(device), weights_only=True)
        return checkpoint

In [ ]:
%cd ShallowWaterJAX

/content/ShallowWaterJAX


In [ ]:
!ls

4DVar.ipynb  model.pth.tar  pytorch_assimilation.py	pytorch_model.py	pytorch_swe_inv.py
data.pkl     project.md     pytorch_emulator_module.py	pytorch_observation.py	README.md


In [ ]:
filename = 'model.pth.tar'
checkpoint = get_checkpoint(filename)

In [ ]:
keys = [x for x in checkpoint['model_state_dict']]
print(keys) # See the keys of all tensors stored in there
for key in keys:
  print(key, checkpoint['model_state_dict'][key].shape)

['dynamics.weight', 'dynamics.bias', 'statics.weight', 'statics.bias', 'encoder.0.conv.0.weight', 'encoder.0.conv.0.bias', 'encoder.0.conv.1.weight', 'encoder.0.conv.1.bias', 'encoder.0.conv.3.weight', 'encoder.0.conv.3.bias', 'encoder.0.conv.4.weight', 'encoder.0.conv.4.bias', 'encoder.1.conv.0.weight', 'encoder.1.conv.0.bias', 'encoder.1.conv.1.weight', 'encoder.1.conv.1.bias', 'encoder.1.conv.3.weight', 'encoder.1.conv.3.bias', 'encoder.1.conv.4.weight', 'encoder.1.conv.4.bias', 'encoder.2.conv.0.weight', 'encoder.2.conv.0.bias', 'encoder.2.conv.1.weight', 'encoder.2.conv.1.bias', 'encoder.2.conv.3.weight', 'encoder.2.conv.3.bias', 'encoder.2.conv.4.weight', 'encoder.2.conv.4.bias', 'encoder.3.conv.0.weight', 'encoder.3.conv.0.bias', 'encoder.3.conv.1.weight', 'encoder.3.conv.1.bias', 'encoder.3.conv.3.weight', 'encoder.3.conv.3.bias', 'encoder.3.conv.4.weight', 'encoder.3.conv.4.bias', 'encoder.4.conv.0.weight', 'encoder.4.conv.0.bias', 'encoder.4.conv.1.weight', 'encoder.4.conv.1.

In [ ]:
from pytorch_model import UNET2D
DEVICE = 'cpu'

network_params = {
    'name': 'UNET2D',
    'condi_net': False,
    'in_channels': [3, 2],
    'out_channels': [3],
    'features': [16, 32, 64, 128, 256],
    'kernel_size': 3,
    'stride': 1,
    'padding': 1,
    'norm_type': 'GroupNorm',
    'activation': 'SiLU',
    'num_layers': 5,
}

model = UNET2D(**network_params)
model = load_model(swe_dxdt, filename).to(DEVICE)

DEVICE on Loading model:  cpu
+++++++++++++++++++ LOADING trained model ++++++++++++++++++++++++++


<ipython-input-21-4f881d23b9a5>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, map_location=torch.device(device))


In [ ]:
print(model)

UNET2D(
  (dynamics): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (statics): Conv2d(2, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder): ModuleList(
    (0): DoubleConv2D(
      (conv): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): GroupNorm(4, 16, eps=1e-05, affine=True)
        (2): SiLU(inplace=True)
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): GroupNorm(4, 16, eps=1e-05, affine=True)
        (5): SiLU(inplace=True)
      )
    )
    (1): DoubleConv2D(
      (conv): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): GroupNorm(8, 32, eps=1e-05, affine=True)
        (2): SiLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): GroupNorm(8, 32, eps=1e-05, affine=True)
        (5): SiLU(inplace=True)
      )
    )
    (2): DoubleConv2D(
   